# Imports

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{max-width:80%!important;width:auto!important;}</style>"))

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
def show_img(im, ax=None, figsize=(8,8)):
    if not ax: _,ax = plt.subplots(1,1,figsize=figsize)
    if len(im.shape)==2: im = np.tile(im[:,:,None], 3) 
    ax.imshow(im[:,:,::-1]);
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    return ax

In [ ]:
def show_imgs(ims, rows=1, figsize=(16,8)):
    _,ax = plt.subplots(rows, len(ims)//rows, figsize=figsize)
    [show_img(im,ax_) for im,ax_ in zip(ims,ax.flatten())]
    return ax

# HOG (Histograms of Oriented Gradients)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm.notebook import trange

In [ ]:
im = cv2.imread('Data/trial.jpeg')
show_img(im);

In [ ]:
im_1 = im[252:380, 210:338].copy()
im_2 = cv2.resize(im[450:610, 402:562], (128,128))

_,ax = plt.subplots(1,2)
show_img(im_1, ax[0]);
show_img(im_2, ax[1]);

im_1.shape, im_2.shape

## Gradients

Gradient calculation using the `Sobel` operator.

In [ ]:
gradX = cv2.Sobel(im_1, ddepth=cv2.CV_32F, dx=1, dy=0) # reveal vertical edges
gradY = cv2.Sobel(im_1, ddepth=cv2.CV_32F, dx=0, dy=1) # reveal horizontal edges

# gradient magnitude and angle/direction
grad = cv2.convertScaleAbs(np.sqrt(gradX**2 + gradY**2))
angles = np.mod(np.arctan(gradY/(gradX+1e-10)) * 180/np.pi + 180, 180)

# grad2, angles2 = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)
# grad2 = cv2.convertScaleAbs(grad2)

show_img(grad, figsize=(6,6));

In [ ]:
ix = np.argmax(grad, axis=2)
I,J = np.indices(ix.shape)

In [ ]:
sr, sc = slice(15,23), slice(87,95)
ax = show_imgs((cv2.rectangle(im_1.copy(), (sc.start,sr.start), (sc.stop,sr.stop), (0,255,0)), im_1[sr,sc]))
ax[1].quiver(gradX[I,J,ix][sr,sc], gradY[I,J,ix][sr,sc], color='red');

## Histogram

We want a descriptor for each 8x8* patch.

\* this is what worked well in the paper that proposed HoG; other sizes are possible

In [ ]:
im_copy = im_1.copy()
for x in range(im_copy.shape[1]//8):
    for y in range(im_copy.shape[0]//8):
        cv2.rectangle(im_copy, (x*8, y*8), ((x+1)*8, (y+1)*8), (0,255,0), 1)

ax = show_img(im_copy)

In [ ]:
grad_mx = grad[I,J,ix]
angles_mx = angles[I,J,ix]

grad_p = grad_mx[sr,sc]
angles_p = angles_mx[sr,sc]

grad_p, angles_p

Our histogram will have 9 bins, 20 degrees apart. Angles above 160 degrees contribute both to the last and first been (wrap around 180), proportionally.

In [ ]:
bins = np.arange(-20,181,20)
bins

In [ ]:
def hog_descr(grad_p, angles_p):
    bin_cnts = np.zeros((11,))

    for g,a in zip(grad_p.flatten(),angles_p.flatten()):

        # which bin(s)?
        b1 = np.argwhere(a>bins).max()
        b2 = np.argwhere(a<bins).min()
    #     print(a,b1,b2,bins[b1],bins[b2])

        if b2-b1==2:
            bin_cnts[(b1+b2)//2] += g
        else:
            w2 = abs(bins[b1]-a) / (bins[b2]-bins[b1])
            w1 = abs(bins[b2]-a) / (bins[b2]-bins[b1])
            bin_cnts[b1] += w1*g
            bin_cnts[b2] += w2*g

    bin_cnts[1] += bin_cnts[-1]
    bin_cnts = bin_cnts[1:-1] # drop bins -20 and 180
    return bin_cnts

In [ ]:
bin_cnts = hog_descr(grad_p, angles_p)

In [ ]:
bin_cnts.sum(), grad_p.sum()

In [ ]:
plt.bar(bins[1:-1], bin_cnts, width=15)
plt.xticks(bins[1:-1]);

In [ ]:
descr = np.zeros((grad.shape[1]//8, grad.shape[0]//8, 9))

for x in trange(grad.shape[1]//8):
    for y in range(grad.shape[0]//8):
        descr[y,x] = hog_descr(grad_mx[y*8:(y+1)*8, x*8:(x+1)*8], angles_mx[y*8:(y+1)*8, x*8:(x+1)*8])

### Normalisation 

Gradient magnitudes depend on lighting; if we changed the brightness of the image, the gradients will change as well - we want to counteract this by normalising the histograms. We could normalise each of the histograms individually but for better results we'll do it in 2x2 blocks (or 16x16 pixel regions).

In [ ]:
descr.shape

In [ ]:
descr_norm = np.zeros_like(descr)
for x in range(descr.shape[1]-1):
    for y in range(descr.shape[0]-1):
        d = descr[y:y+2, x:x+2]
        descr_norm[y:y+2,x:x+2] += d / np.sqrt((d**2).sum())

In [ ]:
dv = np.full(descr_norm.shape[:2], 4)
a = np.array([1] + (descr.shape[0]-2)*[2] + [1])
dv[0, :] = a
dv[-1,:] = a
dv[:, 0] = a
dv[:,-1] = a
dv

In [ ]:
descr_norm /= dv[...,None]
descr_norm[:4,:4]

## HOG in scikit-image

In [ ]:
from skimage.feature import hog
from skimage import data, exposure

fd, hog_image = hog(im, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=True)
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

_, ax = plt.subplots(1, 2, figsize=(16, 8), sharex=True, sharey=True)

ax[0].axis('off'); ax[0].imshow(im[:,:,::-1]);
ax[1].axis('off'); ax[1].imshow(hog_image_rescaled, cmap=plt.cm.gray);